In [3]:
import pandas as pd

data = pd.read_csv("train.csv",usecols=['Sex'])
data.head()

,Sex
0,male
1,female
2,female
3,female
4,male


In [ ]:
#I want to predict a passenger is survive or not based on the lot's of fact

#-----------------------------Simple one i am using one hot encoding Variable

'''
Pros - Easy To use No Extra Information Required.
Cons - We have multiple category in a variable then we will endup with Thousand of columns so our data will become more sparse. 
'''

In [4]:
# One Hot Encoding

pd.get_dummies(data).head()

,Sex_female,Sex_male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [5]:
#For Better Visualization

pd.concat([data,pd.get_dummies(data)],axis=1).head()

,Sex,Sex_female,Sex_male
0,male,0,1
1,female,1,0
2,female,1,0
3,female,1,0
4,male,0,1


In [7]:
#we always drop one variable
pd.get_dummies(data,drop_first=True).head()

,Sex_male
0,1
1,0
2,0
3,0
4,1


In [12]:
#---------------------this exaple is more then two labels
data = pd.read_csv("train.csv",usecols=['Embarked'])
data.head()

,Embarked
0,S
1,C
2,S
3,S
4,S


In [13]:
pd.get_dummies(data).head()

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [15]:
pd.get_dummies(data,drop_first=True).head()

,Embarked_Q,Embarked_S
0,0,1
1,0,0
2,0,1
3,0,1
4,0,1


In [16]:
#If We have missing categorical Variable So we have to add one argument
pd.get_dummies(data, drop_first=True, dummy_na=True).head()

,Embarked_Q,Embarked_S,Embarked_nan
0,0,1,0
1,0,0,0
2,0,1,0
3,0,1,0
4,0,1,0


In [17]:
pd.get_dummies(data, drop_first=True, dummy_na=True).sum(axis=0)

Embarked_Q       77
Embarked_S      644
Embarked_nan      2
dtype: int64

In [18]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

C:\Users\AITeam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [19]:
#Read The DataSets
data = pd.read_csv('train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
#Make A Copy of Dataset So we will use this Concept Furture
data_OHE = pd.concat([data[['Pclass', 'Age', 'SibSp','Parch', 'Survived']], # numerical variables
                    pd.get_dummies(data.Sex, drop_first=True),   # binary categorical variable
                    pd.get_dummies(data.Embarked, drop_first=True)],  # k categories in categorical
                    axis=1)

data_OHE.head()

,Pclass,Age,SibSp,Parch,Survived,male,Q,S
0,3,22.0,1,0,0,1,0,1
1,1,38.0,1,0,1,0,0,0
2,3,26.0,0,0,1,0,0,1
3,1,35.0,1,0,1,0,0,1
4,3,35.0,0,0,0,1,0,1


In [23]:
#And Now Let's Seprate into Training Set And Test Sets
X_train, X_test, y_train, y_test = train_test_split(data_OHE[['Pclass', 'Age', 'SibSp',
                                                              'Parch', 'male', 'Q', 'S']].fillna(0),
                                                    data_OHE.Survived,
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((623, 7), (268, 7))

In [24]:
# let's build a random forest

rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=3)
rf.fit(X_train, y_train)
print('Train set')
pred = rf.predict_proba(X_train)
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = rf.predict_proba(X_test)
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
Random Forests roc-auc: 0.8696177957094205
Test set
Random Forests roc-auc: 0.8606250000000001


In [25]:
# and now an AdaBoost classifier

ada = AdaBoostClassifier(n_estimators=200, random_state=44)
ada.fit(X_train, y_train)
print('Train set')
pred = ada.predict_proba(X_train)
print('AdaBoost roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = ada.predict_proba(X_test)
print('AdaBoost roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
AdaBoost roc-auc: 0.9010433613153728
Test set
AdaBoost roc-auc: 0.8474404761904762


In [26]:
# and finally a logistic regression

logit = LogisticRegression(random_state=44)
logit.fit(X_train, y_train)
print('Train set')
pred = logit.predict_proba(X_train)
print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = logit.predict_proba(X_test)
print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
Logistic Regression roc-auc: 0.8452745059760092
Test set
Logistic Regression roc-auc: 0.8425595238095238


In [ ]:
'''
One Hot Encoding -
    Advantages -
        Straightforword to implement
    Disadvantage -
        if we have lot's of variable We can't Implement.
'''

In [28]:
#Take the Cabin dataSet
data = pd.read_csv('train.csv', usecols = ['Cabin'])
data.head()

,Cabin
0,NaN
1,C85
2,NaN
3,C123
4,NaN


In [29]:
print('number of labels: {}'.format(len(data.Cabin.unique())))

number of labels: 148


In [30]:
Cabin_OHE = pd.get_dummies(data.Cabin)
Cabin_OHE.shape

(891, 147)

In [31]:
Cabin_OHE.head()

,A10,A14,A16,A19,A20,A23,A24,A26,A31,A32,...,E8,F E69,F G63,F G73,F2,F33,F38,F4,G6,T
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
data = pd.read_csv("train.csv")

In [57]:
#I will Do One Experiment Here Let's See it's Work Or Not
data_OHE1 = pd.concat([data[['Pclass', 'Age', 'SibSp','Parch', 'Survived']], # numerical variables
                    pd.get_dummies(data.Sex, drop_first=True),   # binary categorical variable
                    pd.get_dummies(data.Embarked, drop_first=True),
                    pd.get_dummies(data.Cabin,drop_first=True)],  # k categories in categorical
                    axis=1)

data_OHE1.head()


,Pclass,Age,SibSp,Parch,Survived,male,Q,S,A14,A16,...,E8,F E69,F G63,F G73,F2,F33,F38,F4,G6,T
0,3,22.0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,38.0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,26.0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,35.0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,35.0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
data1 = data_OHE1['Survived']
data2 = data_OHE1.drop('Survived',axis='columns')

In [59]:
data2.head()

,Pclass,Age,SibSp,Parch,male,Q,S,A14,A16,A19,...,E8,F E69,F G63,F G73,F2,F33,F38,F4,G6,T
0,3,22.0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,38.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,26.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,35.0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,35.0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
#And Now Let's Seprate into Training Set And Test Sets
X_train, X_test, y_train, y_test = train_test_split(data2.fillna(0),
                                                    data1,
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((623, 153), (268, 153))

In [63]:
# let's build a random forest

rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=3)
rf.fit(X_train, y_train)
print('Train set')
pred = rf.predict_proba(X_train)
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = rf.predict_proba(X_test)
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
Random Forests roc-auc: 0.8688423244615084
Test set
Random Forests roc-auc: 0.8493154761904762


In [64]:
# and now an AdaBoost classifier

ada = AdaBoostClassifier(n_estimators=200, random_state=44)
ada.fit(X_train, y_train)
print('Train set')
pred = ada.predict_proba(X_train)
print('AdaBoost roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = ada.predict_proba(X_test)
print('AdaBoost roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
AdaBoost roc-auc: 0.9384286674909438
Test set
AdaBoost roc-auc: 0.8063095238095238


In [65]:
# and finally a logistic regression

logit = LogisticRegression(random_state=44)
logit.fit(X_train, y_train)
print('Train set')
pred = logit.predict_proba(X_train)
print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = logit.predict_proba(X_test)
print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
Logistic Regression roc-auc: 0.865268649270081
Test set
Logistic Regression roc-auc: 0.8458333333333334


In [ ]:
#I am Not Sure How Much Impact It will Give but You should Check With This Senario.